
# Una introducción a `BeatifulSoup`
### Germán Rosati (IDAES-UNSAM, CONICET, PIMSA)

---



La idea de esta práctica es introducir algunos elementos básicos para la utilización de la librería `BeautifulSoup`. Para ello, vamos a seguir algunos ejemplos del libro _Web Scraping with Pyhton_ de Ryan Mitchell.

![alt text](https://cv02.twirpx.net/2545/2545056.jpg?t=20180604011432)

## Scrapeando un sitio simple
---
Empecemos por un sitio no demasiado complejo. 

1. Importamos algunas funciones de la librería urllib.
2. `html = urlopen("http://pythonscraping.com/pages/page1.html")` abrimos una conexión con la página web en cuestión)
3. `print(html.read())`, mostramos el contenido

In [1]:
from urllib.request import urlopen
html = urlopen("http://pythonscraping.com/pages/page1.html")
print(html.read())

b'<html>\n<head>\n<title>A Useful Page</title>\n</head>\n<body>\n<h1>An Interesting Title</h1>\n<div>\nLorem ipsum dolor sit amet, consectetur adipisicing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.\n</div>\n</body>\n</html>\n'


Vemos que es un tanto incomprensible. Es una parva de código html. Para poder hacer un poco más itnerpretable (y utilizable) vamos a recurrir a la libreria [`BeatifulSoup`](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).


El objeto más comúnmente usado en BeautifulSoup es el `BeautifulSoup object`. Veamos cómo funciona.

In [2]:
from bs4 import BeautifulSoup
html = urlopen("http://www.pythonscraping.com/pages/page1.html")
bsObj = BeautifulSoup(html.read())
print(bsObj.h1)

<h1>An Interesting Title</h1>


Lo que hacemos, entonces, es importar la libreríía urlopen y llamar al método `html.read()` para obtener el contenido en html. Luego, transformamos ese objeto en `BeautifulSoup object` que lo transforma a esta estructura:

```{html}
<html><head>...</head><body>...</body></html> => HTML
  <head><title>A Useful Page<title></head>  => HEAD
    <title>A Useful Page</title>  => TITLE
  <body><h1>An Int...</h1><div>Lorem ip...</div></body> => BODY
      <h1>An Interesting Title</h1>  => H1
      <div>Lorem Ipsum dolor...</div>  => DIV
```

Notar que el tag `<h1>` extraído estaba anidado dos capas dentro del objeto bs.(html → body → h1). 

Sin embargo, cuando lo trajimos, simplemente lo llamamos:

`bsObj.h1`

En realidad, cualquiera de las siguientes llamadas producirá el mismo ouput:


In [3]:
bsObj.html.body.h1 ==  bsObj.body.h1 == bsObj.html.h1 == bsObj.h1

True

En líneas generales, casi cualquier información puede ser extraída de un archivo html a condición de que se encuentren cerca de algún tag que permita identificarlo.

## Parseando un html más complejo
---

Poder mandarse de cabeza tan fácilmente en la estructura del html parece seductor. De hecho, uno podría pensar en generar sentencias cómo éstas:

```python
bsObj.findAll("table")[4].findAll("tr")[2].find("td").findAll("div")[1].find("a")
```

- Poco legible
- Muy vulnerable al cambio en los sitios que se scrapean

Veamos de qué forma podemos tratar de escribir algunos scrapers más prolijos y un poco más robustos.

En general, casi el 100% de los websites contienen "hojas de estilo". Lo interesante es que esas hojas de estilo (codeadas en un estilo que se llama CSS) permiten taggear las diferentes partes del código html para darles un estilo diferente. Por ejemplo:

```{html}
<span class="green"></span>
<span class="red"></span>
```

Los scrapers pueden separar de forma muy simple esos dos tags basados en su atributo `class`. Por ejemplo, podríamos traer todo el texto escrito en rojo. 

Veamos ahora este [sitio](http://www.pythonscraping.com/pages/warandpeace.html).

In [0]:
html = urlopen("http://www.pythonscraping.com/pages/warandpeace.html")
bsObj = BeautifulSoup(html)

Traigamos ahora, todas las palabras en verde.

In [5]:
nameList = bsObj.findAll("span", {"class":"green"})
for name in nameList:
  print(name.get_text())

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


## Mini práctica
---
Ahora traigan ustedes todas las palabras en rojo

In [0]:
# INSERTE EL CODIGO

nameList = bsObj.findAll("span", {"class":"red"})

name_1 = []
for name in nameList:
  name_1.append(name.get_text())

### Una aclaración con `get_text()`

- `.get_text()` elimina todos los tags del documento y devuelve un string que solamente contiene texto. Por ejemplo, si están trabajando con un bloque largo de texto que tiene links, imágenes, etc... todo eso desaparece y solo devuelve un bloque de texto. 
- Es importante recordar que es mucho más fácil buscar cosas en el árbol del objeto `BeautifulSoup` que en texto plano.
- `.get_text()` debería ser usando cuando estén seguros de que ya no buscan nada más en el árbol.

### Quizás las dos funciones más útiles: `find() y find_all()`
---

Esas son las dos funciones que, seguramente, más van a usar. Permiten filtrar de forma bastante simple pááginas html para encontrar listas de tags buscados o un solo tag, basados en sus atributos. Son muy similares en su sintaxis y en sus argumentos:

```{python}
findAll(tag, attributes, recursive, text, limit, keywords)
find(tag, attributes, recursive, text, keywords)
```
En la mayoría de los casos van a necesitar solamente los dos primeros argumentos de la función: el tag buscado y sus atributos.

- `tag`: pasamos un string con el nombre del tag que buscamos; por ejemplo, la línea siguiente va a traer todos los headers en el documento: `.findAll({"h1","h2","h3","h4","h5","h6"})`
- `attributes`: toma un diccionarii de atributos y matchea tags que contiene cualquiera de esos atributos; por ejemplo, `.findAll("span", {"class":"green", "class":"red"})` va a devolver los tags con `class=green` y `class=red`
- `recursive`: es un booleano, que especifica cuál es la profundidad con la que se desea navegar el árbol. Si `recursive==True` la función busca hasta en los niveles más bajos tags que matcheen. Si se setea `recursive==False` solamente buscará en los tags de los nieles máás altos.
- `text`: no es muy usual y simplemente matchea basado en el texto contenido en el tag, en lugar de matchear los tags mismos. Por ejemplo, si quisiéramos ver cuántas veces aparece rodeado de tags el término "the prince" en la página anterior podríamos hacer los siguiente:

In [13]:
nameList = bsObj.findAll(text="the prince")
print(len(nameList))

7


## Problemas de conectividad
---
Inevitablemente nos vamos a encontrar en algún momento con algún problema de conectividad. Vamos a querer descargar algún html y las cosas van a fallar. Más de una vez, hemos dejado corriendo un scraper toda la noche y resulta que el programa se cortó a la segunda iteración porque no logró bajar alguno de los links que pensábamos descargar.

No obstante, hay algunas formas de evitar estos problemas. Volvamos a nuestra primera línea

In [0]:
html = urlopen("http://www.pythonscraping.com/pages/page1.html")

Hay (básicamente) dos cosas que pueden fallar acá:

- la página no está en el servidor
- el servidor no se encuentra

En el primer caso, nos va a devolver un `HTTPError` Este HTTP error puede ser “404 Page Not Found,” “500 Internal Server Error,” etc. En todos los casos la función `urlopen` va a tirar `HTTPError`. Podemos manejar ese error de la 
siguiente forma:

```python
try:
  html = urlopen("http://www.pythonscraping.com/pages/page1.html")
except HTTPError as e:
  print(e)
  #devuelve null, break, or hace alguna otra cosa
else:
  #el programa continua
```

---
**Aclaración:** este uso de `try` y `except` forma parte de lo que se denomina manejo de excepciones (exception handling). Si bien queda fuera del alcance de esta clase pueden encontrar material en los siguientes links:

- [documentación al respecto de Python](https://docs.python.org/3/tutorial/errors.html)
- [un lindo tutorial sobre el tema](https://realpython.com/python-exceptions/)
---

Si aparece un HTTP error el programa ahora imprime el error y no ejecuta todo lo que está debajo del `else`.

Si el problema es que no encuentra el servidor, por ejemplo, que el sitio estuviera mal escrito o caído, `urlopen` devuelve un `None`, algo así como un `NULL`, podemos, entonces, hacer un test para ver si lo que devolvió es `None`:


```python
if html is None:
  print("URL is not found")
else:
# el programa sigue
```

Ahora bien, incluso si trajimos la página correctamente desde el server, todavía puede suceder que el sitio no esté formateado como lo esperamos. Suele ser una buena práctica chquear que el tag que buscamos existe... si no existe, `BeatifulSoup` devuelve un `None`.

In [15]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup

def getTitle(url):
  try:
    html = urlopen(url)
  
  except HTTPError as e:
    return None
  try:
    bsObj = BeautifulSoup(html.read())
    title = bsObj.body.h1
  except AttributeError as e:
    return None
  return title

title = getTitle("http://www.pythonscraping.com/pages/page1.html")
if title == None:
  print("Title could not be found")
else:
  print(title)

<h1>An Interesting Title</h1>


Aquí creamos una función `getTitle` qye devuelve o bien el título de la página o un objeto `None` si hay algún problema. Dentro de la funciín se chequea si existe algún `HTTPError` y también encaspula en un solo `try` dos líneas de BeatifulSoup.

Un `AttributeError` puede aparecer desde cualquiera de estas líneas (si el servidor no existiera, entonces el objeto html sería un `None` y html.read () arrojaría un `AttributeError`). 

Podríamos, de hecho, abarcar tantas líneas como quisiéramos dentro de `try`, o llamar a otra función por completo, lo que puede arrojar un `AttributeError` en cualquier momento.

Al escribir scrapers, es importante pensar en el patrón general de su código
para manejar excepciones y hacerlo legible al mismo tiempo. También es probable
desea reutilizar mucho el código. Tener funciones genéricas como `getSiteHTML` y `getTitle` (con buen manejo de expeciones) es una buena práctica.


## Resumen
---

Vimos algunas cuestiones básicas de scraping:

- parseo de sitios html simples
- algunas funciones y métodos de `BeautifulSoup`
  - `urlopen()`
  - `BeautifulSoup()`
  - `find()` y `find_all()`
  - `.get_text()`
  - manejar excepciones con `try` y `except`

Nos quedan unas cuántas cosas afuera en esta introducción. Solo por citar algunas:

1. cómo navegar el árbol hacia arriba y hacia abajo (es decir los `children`, `siblings` y `parents`
2. Regex (o regular expressions) que, seguramente, verán en algún momento del curso
3. Como scrapear sitios que no son estáticos (por ejemplo que tienen formularios, etc., o paginado, etc.)

Para los 3 pueden recurrir al libro  [Web Scraping with Pyhton de Ryan Mitchell](https://www.oreilly.com/library/view/web-scraping-with/9781491985564/) y/o a la documentación de [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/).

Para el último punto, además, les dejo dos paquetes que son óptimos para este tipo de tareas, aunque un poco más complejos de usar:

- [Selenium](https://selenium-python.readthedocs.io/)
- [Scrapy](https://scrapy.org/)

# APIS
---
Una consulta a una API (Applications Programming Interface) puede servir para  resolver muchos de los problemas que estuvimos viendo al momento de parsear un html. En efecto, proveen una interface conveniente y sumple para muchas aplicaciones. No importa si las aplicaciones fueron escritas por diferentes programadores, con diferentes arquitecturas... la idea de una API es servir como lingua franca entre diferentes pedazos de software para compartir información.

## Cómo funciona una API
---
Si bien no son tan generalizadas como uno querría, suele haber una cantidad de APIS bastante grande y con diferentes tipos de información. Data de músicos y músoca (Spotify), de lugares (Google Places o OSM), de deportes (ESPN), de pobreza (Banco Mundial), etc.

Las APIs son muy sumples de usar...

Copien y peguen esta direccion en su navegador:

> [https://freegeoip.live/json/50.78.253.58](https://freegeoip.live/json/50.78.253.58)

Esto producirá la siguiente respuesta:

```{json}
{"ip":"50.78.253.58","country_code":"US","country_name":"United States","region_code":"MA","region_name":"Massachusetts","city":"Fitchburg","zip_code":"01420","time_zone":"America/New_York","latitude":42.5781,"longitude":-71.8051,"metro_code":506}
```

Es decir, navegamos a una dirección web en el navegador y ese navagador poroduce cierta información... que adeḿas (aunque no lo parezca) esta MUY BIEN FORMATEADA.

¿Qué diferencia hay con los sitios web normales? Bastante poca. Usan el mismo protocolo (HTTP). Lo único ciertamente distintivo es que las APIs tienen una sintaxis extremadamente regulada y que las APIs tienden a presentar sus datos en formato JSON o XML y no en HTML.


## Trayendo datos de pobreza
---
Veamos un ejemplo un poco más complejo. [PovcalNet](http://iresearch.worldbank.org/PovcalNet/home.aspx) es un proyecto del Banco Mundial. Se trata de una herrmienta que permite replicar los cálculos que el organismo realiza sobre la llamada "Pobreza Absoluta" para una gran grupo de paises en el mundo. No nos vamos a meter en detalles pero la cosa es que hay dos grandes formas de medir la pobreza: [la absoluta y la relativa](https://www.habitatforhumanity.org.uk/blog/2018/09/relative-absolute-poverty/). La pobreza absoluta define un cierto umbral de ingresos por medio del cual puede adquirirse una canasta mínima de bienes y servicios. Aquellos hogares que se encuentren por debajo de ese umbral serán clasificados omo "pobres". Ahora bien, las canastas varían notablemente entre países por diversos motivos (históricos, culturales, metodológicos) es por ello que el Banco Mundial definió el criterio de los "2 dólares diarios": cualquier persona que no llegue a ese ingreso es considerado como "pobre extremo".

Lo interesante (entre otras cosas) de este proyecto es que no te hace bajar un mamotreto de datos sino que... tiene una API, bastante completa para trabajar. De hecho, podemos jugar con diferentes umbrales de pobreza absoluta. Si U$S nos parece demasiado bajo podemos utilizar otro umbral más alto y la API nos va a devolver cálculos de pobreza para dichos umbrales.


> **Dato anecdótico (y no tanto):** si consultan el sitio de PovcalNet van a encontrar varios documentos metodológicos. Además, van a encontar la "historia" de los famososo U$$2 diarios. Básicamente, se hizo un estudio sobre los 15 países con menor gasto de consumo personal per cápita del mundo (Malawi, Mali, Ethiopia, Sierra Leone, Niger, Uganda, Gambia, Rwanda, Guinea-Bissau, Tanzania, Tajikistan, Mozambique, Chad, Nepal y Ghana). Es decir, que se trata de un umbral bastante bajo...


Veamos una introducción a esta API. Vamos a trabajar sobre una partecita... la puntita del iceberg de este proyecto. [Acá](http://iresearch.worldbank.org/PovcalNet/docs/PovcalNet%20API.pdf) tienen la documentación completa de la API.

*Nota: intenté calcular los mísmos índices para Argentina pero devuelve un conjunto vacío... raro*

En líneas generales, esto es una consulta sintácticamente correcta a la API:

> http://iresearch.worldbank.org/PovcalNet/PovcalNetAPI.ashx?C0=USA&PL0=2.5&Y0=2000&format=json'

Tiene varias partes:

- el protocolo: `http://`,
- el nombre del servidor: `iResearch.worldbank.org`,
- el nombre del sitio: `PovcalNet`,
- el "handler": `PovcalNetAPI.ashx`,
- el string de la query: `C0=USA&PL0=2.5&Y0=2000`,
- el formato de respuesta requerido: `format=json`.

Centrémonos en los dos últimos;

- `C0=USA` es el parámetro de país... en este caso, los Estados Unidos
- `PL0=2.5` refiere al umbral de los "dólares diarios". Quienes ganen menos (en este caso de U$S2.5 por día serán considerados como pobres.
- `Y0=2000` es el año de estimación

Si hacen click en el enlacec verán que devuelve una respuesta en formato json con varios campos. Nos vamos a centrar en el que se llama `HC` que es simplemente el conteo en proprociones de la cantidad de personas que se encuentran por debajo del umbral. En este caso, son 0.007, es decir, menos del 1% de la poblacion en USA. 


Hagamos una consulta más compleja. Primero, importamos las dos librerías que vamos a usar...

In [0]:
import pandas as pd
import requests

Vamos a hacer una consulta al sitio que nos devuelva

- datos de USA `C0=USA`
- con línea de pobreza de U$S1.9 `pl=1.9`
- para los años 1991 y 2002 `Y0=1991,2000`

Vamos a generar los parámetros en objetos y usar los [`f.string` de Python](https://realpython.com/python-f-strings/) para construir la url para hacer la consulta a la API:

In [0]:
country='USA'
pl='1.9'
year='1991,2000'
form='json'

url = f'http://iresearch.worldbank.org/PovcalNet/PovcalNetAPI.ashx?C0={country}&PL0={pl}&Y0={year}&format={form}'

Veamos si la URL quedó bien escrita:

In [0]:
url

'http://iresearch.worldbank.org/PovcalNet/PovcalNetAPI.ashx?C0=USA&PL0=1.9&Y0=1991,2000&format=json'

Luego, usamos el paquete requests para traer la url, pasarla a formato json y pandas para pasarla a un formato de dataframe:

In [0]:
req = requests.get(url).json()['PovResult']
req = pd.DataFrame.from_dict(req)
req

,interpolated,useMicroData,CountryCode,CountryName,RegionCode,CoverageType,RequestYear,DataYear,DataType,PPP,PovertyLine,Mean,HC,pg,PovGapSqr,Watts,Gini,Median,pr.mld,Polarization,ReqYearPopulation,SvyInfoID,Decile
0,False,True,USA,United States,OHI,3,1991,1991,Income,1,1.9,1623.643292,0.004990,0.003493,0.002820,0.010560,0.3824,1326,0.2793,-1,252.98,USA_N1991Y,"[0.01905, 0.03571, 0.04867, 0.06147, 0.07452, ..."
1,False,True,USA,United States,OHI,3,2000,2000,Income,1,1.9,1906.026841,0.007492,0.005991,0.005259,0.009052,0.4034,1483,0.3189,-1,282.16,USA_N2000Y,"[0.01869, 0.03569, 0.04748, 0.05908, 0.07121, ..."


## Mini actividad
---
Traer estimaciones de pobreza desde povcalnet para los Tanzania con un umbral de pobreza de U$S1.9 (los famosos "dos dólares diarios") para los años 1991 y 2000. ¿Cuál es la proporción de pobres según este criterio para cada año?

Tip: el código de Tanzania en PovcalNet es TZA

In [0]:
## INSERTE CODIGO AQUI


¿Qué pueden decir las proporiones de pobres extremos en Tanzania y USA?

## Autenticación
---

Muchas veces las API requieren algún mecanismo de registro para poder usarlas. Esto puede ser por mera seguridad o para cobrar por su uso.

La gran mayoría de los métodos de autenticación de API generalmente usan algún tipo de token que se pasa al servidor web con cada llamada API realizada. El token es proporcionado al usuario cuando el usuario se registra y es un elemento permanente de la llamadas del usuario (generalmente en aplicaciones de baja seguridad), o puede cambiar con frecuencia, y se recupera del servidor utilizando una combinación de nombre de usuario y contraseña.

Por ejemplo, para hacer una consulta al Sistema Integrado de Información Sanitaria Argentina del Ministerio de Salud sobre los establecimientos de salud, deberíamos escribir algo como esto:

`https://sisa.msal.gov.ar/sisa/services/rest/establecimiento/buscar?provincia=1&redEstablecimiento=5Post:{"usuario":"jperez","clave":"xxxx"}`

Usando urllib podríamos tratar de hacer una consulta:

```python
usuario = "<tu user>"
clave = "<tu clave>"
webRequest = urllib.request.Request("http://myapi.com", headers={"usuario":token, "clave": clave})
html = urlopen(webRequest)
```

De cualquier forma, en este tutorial vamos a usar una librería de Python que nos va a facilitar el manejo de la autenticación.